In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry

Generate and Save Input Data - Only run once.

In [ ]:
target_dt = 1.0/30

data = pd.read_csv("multidata.csv", low_memory=False)
x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
labs = np.array(data.iloc[:,443]).astype(str) #array of labels (purple child annotations, engagement) 218/443
a = []
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    if i == 180:               # add these three lines
        y[y>0]-=np.pi*2   # add these three lines
        y += np.pi            # add these three lines
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T
    
def extract_pattern(start, end, target_dt): #function to extract consecutive frames with the same label 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)
    timestamp = np.array(data.iloc[start:end,0]).astype(str)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    t = np.arange(int(t_sample[-1]/target_dt))*target_dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return timestamp, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])

t_noplay=[]
p_noplay=[]
t_goal=[]
p_goal=[]
t_aim=[]
p_aim=[]

for i in range(1,(len(start)-1)):
    try:
        if label[i]==('noplay'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_noplay.append(ts)
            p_noplay.append(pi)
        if label[i]==('goaloriented'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_goal.append(ts)
            p_goal.append(pi)
        if label[i]==('aimless'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_aim.append(ts)
            p_aim.append(pi)
    except IndexError:
        print('empty pattern')
        
i=0
goal = []
noplay = []
aim = []
for entry in t_goal:
    goal.append([t_goal[i],p_goal[i]])
    i = i+1
i=0
for entry in t_noplay:
    noplay.append([t_noplay[i],p_noplay[i]])
    i = i+1
i=0
for entry in t_aim:
    aim.append([t_aim[i],p_aim[i]])
    i = i+1
    
np.save('goal.npy', goal)
np.save('noplay.npy', noplay)
np.save('aim.npy', aim)

Set up network.

In [8]:
class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        self.param('length of pattern', len_train=10),
        self.param('number of epochs', n_epoch=1),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        goal = list(np.load('goal.npy'))
        noplay = list(np.load('noplay.npy'))
        aim = list(np.load('aim.npy'))
        
        n_dims=2 
        seed=param.p_seed

        dt = 0.001
        target_dt = 1.0/30
        D = n_dims # param.n_dims
        classify_score = {}
        accuracy = {}

        p_goal2 = random.sample(goal, len(goal))
        p_noplay2 = random.sample(noplay, len(noplay))
        p_aim2 = random.sample(aim, len(aim))
        p_aim2 = np.array(p_aim2)

        goal_train_ts = np.array(p_goal2[:(int(len(p_goal2)*0.8))]) 
        goal_test_ts = np.array(p_goal2[(int(len(p_goal2)*0.8)):]) 
        goal_train_list=list(goal_train_ts[:,1])
        goal_test_list=list(goal_test_ts[:,1])
        goal_train_time=list(goal_train_ts[:,0])
        goal_test_time=list(goal_test_ts[:,0])

        noplay_train_ts = np.array(p_noplay2[:(int(len(p_noplay2)*0.8))]) 
        noplay_test_ts = np.array(p_noplay2[(int(len(p_noplay2)*0.8)):]) 
        noplay_train_list=list(noplay_train_ts[:,1])
        noplay_test_list=list(noplay_test_ts[:,1])
        noplay_train_time=list(noplay_train_ts[:,0])
        noplay_test_time=list(noplay_test_ts[:,0])
        
        for p in noplay_train_list:
            if p.size > 0:
                pass
            else:
                noplay_train_list.remove(p)

        for p in noplay_test_list:
            if p.size >0:
                pass
            else:
                noplay_test_list.remove(p)

        train_all = np.vstack(goal_train_list+noplay_train_list)
        pca_model = PCA(n_components=D).fit(train_all)

        train_time_all = np.hstack(goal_train_time+noplay_train_time)
        test_time_all = np.hstack(goal_test_time+noplay_test_time)

        goal_train_pca = np.vstack([pca_model.transform(p) for p in goal_train_list])
        noplay_train_pca = np.vstack([pca_model.transform(p) for p in noplay_train_list])

########## TESTING PATTERNS - ONE SEGMENT AT A TIME #########
        goal_test_pca2=[]
        noplay_test_pca2=[]

        for i in range(len(goal_test_list)):
            goal_test_pca2.append(pca_model.transform(goal_test_list[i]))

        for i in range(len(noplay_test_list)):
            noplay_test_pca2.append(pca_model.transform(noplay_test_list[i]))

#############################################################

        T_train = param.len_train   # number of seconds to train on for each class
        T_test = 100    # number of seconds to test on for each class
        
        goal_train_time2 = [item for sublist in goal_train_time for item in sublist]
        noplay_train_time2 = [item for sublist in noplay_train_time for item in sublist]

        #goal_test_time2 = [item for sublist in goal_test_time for item in sublist]
        #noplay_test_time2 = [item for sublist in noplay_test_time for item in sublist]

        N_frames = int(T_train*30)
        #training = np.vstack([goal_train_pca[:N_frames], noplay_train_pca[:N_frames]])
        train_timestamps = np.hstack([goal_train_time2[:(N_frames*param.n_epoch)], noplay_train_time2[:(N_frames*param.n_epoch)]])
        #assert len(training) == N_frames*2

        #N_frames = int(T_test*30)
        #testing = np.vstack([goal_test_pca[:N_frames], noplay_test_pca[:N_frames]])
        #test_timestamps = np.hstack([goal_test_time2[:N_frames], noplay_test_time2[:N_frames]])
        #assert len(testing) == N_frames*2
        
############ TRAINING WITH 80% ############ 
        print('Run Number: ', int(param.p_seed)+1)
        print('Testing with 80% Training Data')
    
        num_epochs = param.n_epoch
        batch_size = int(T_train*30)
        batch = 0
        theta = 15

        for epoch in range(num_epochs):
            training = np.vstack([goal_train_pca[batch:int(batch+batch_size)], noplay_train_pca[batch:int(batch+batch_size)]])
            batch += batch_size

            net = nengo.Network(seed=seed)#param.seed)
            with net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))


                node_pool = nengo.Node(None, size_in=rw[0].state.size_out*D)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, node_pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(training, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_node_pool = nengo.Probe(node_pool, sample_every = 0.1)


            sim = nengo.Simulator(net)
            with sim:
                sim.run(T_train*2)  


            pool_model = nengo.Network()
            with pool_model:
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)
            pool_sim = nengo.Simulator(pool_model)

            import nengo.utils.ensemble

            _, a = nengo.utils.ensemble.tuning_curves(pool, pool_sim, inputs=sim.data[p_node_pool])

            if epoch == 0:
                a_goal = a[:(int(len(a)/2))]
                a_noplay = a[(int(len(a)/2)):]
            else:
                a_goal = np.concatenate((a_goal, a[:(int(len(a)/2))]))
                a_noplay = np.concatenate((a_noplay, a[(int(len(a)/2)):]))

        a_out = np.vstack([a_goal, a_noplay])


        N = int((len(a_out))/2) #int(T_train*1000)
        target = np.hstack([np.ones(N), -np.ones(N)]).reshape(-1, 1)
        dec, info = nengo.solvers.LstsqL2(reg=0.1)(a_out, target)
        
        v = np.dot(a_out, dec)


        train_timestamps = train_timestamps[0::2]

        output_goal_train = {"output_goal_train":{"time":train_timestamps[:N], "output":v[:N]}}
            
        output_noplay_train = {"output_noplay_train":{"time":train_timestamps[N:], "output":v[N:]}}

############ TESTING WITH 20% ############ 
        print('Testing with 20% Testing Data')
        a_goal=[]

        #for j in range(int(len(goal_test_pca2))):
        for j in range(3):
            print('Testing Goal Pattern ', int(1+j))
            testing = np.vstack([goal_test_pca2[j]])

            test_net = nengo.Network(seed=seed)#param.seed)
            with test_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                goal_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(test_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[goal_test_result]

            a_goal.append(v)
            
        output_goal_test = {}
        for i in range(len(a_goal)):
            output_goal_test[i]={"time":goal_test_time[i], "output":a_goal[i]}

        for j in range(len(output_goal_test)):        
            key_j = 'output_goal_test{}'.format("%03d" % j)  
            output_goal_test[key_j] = output_goal_test.pop(j) 
            
############################   

        a_noplay=[]
        #for j in range(int(len(noplay_test_pca2))):
        for j in range(3):
            print('Testing No Play Pattern ', int(1+j))
            testing = np.vstack([noplay_test_pca2[j]])

            test_net = nengo.Network(seed=seed)#param.seed)
            with test_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                noplay_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(test_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[noplay_test_result]

            a_noplay.append(v)
    
        #classify1 = np.isclose(v[:N], 1, atol=0.5)
        #classify2 = np.isclose(v[N:], -1, atol=0.5)
        #classify = np.append(classify1, classify2)
        #score_test = np.mean(classify)

        #classify_score[1]=score_test
            
        #test_timestamps = test_timestamps[0::2]

        output_noplay_test = {}
        for i in range(len(a_noplay)):
            output_noplay_test[i]={"time":noplay_test_time[i], "output":a_noplay[i]}

        for j in range(len(output_noplay_test)):        
            key_j = 'output_noplay_test{}'.format("%03d" % j)  
            output_noplay_test[key_j] = output_noplay_test.pop(j) 

############ TESTING WITH AIMLESS PATTERNS ############ 
        print('Testing with Aimless Data')
    
        aim_list=list(p_aim2[:,1])
        aim_time=list(p_aim2[:,0])
        #aim_time_all = np.hstack(aim_time)

        aim_test_pca2=[]

        for i in range(len(aim_list)):
            aim_test_pca2.append(pca_model.transform(aim_list[i]))

        print('Testing with Aimless Data')
        a_aim=[]

        #for j in range(len(aim_test_pca2)):
        for j in range(3):
            print('Testing Aimless Pattern ', int(1+j))
            testing = np.vstack([aim_test_pca2[j]])

            aim_net = nengo.Network(seed=seed)#param.seed)
            with aim_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                aim_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(aim_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[aim_test_result]

            a_aim.append(v)

        output_aimless = {}
        for i in range(len(a_aim)):
            output_aimless[i]={"time":aim_time[i], "output":a_aim[i]}

        for j in range(len(output_aimless)):        
            key_j = 'output_aimless{}'.format("%03d" % j)  
            output_aimless[key_j] = output_aimless.pop(j) 
            
############ SAVE DATA ############ 
        #classify_score['classify_train'] = classify_score.pop(0)
        #classify_score['classify_test'] = classify_score.pop(1)
        #classify_score['classify_aim'] = classify_score.pop(2)
        
        results = {**output_goal_train, **output_noplay_train, **output_goal_test, **output_noplay_test, **output_aimless}
        #accuracy_output = {**accuracy_train, **accuracy_test, **accuracy_aim}
    
        #results = {**classify_score, **network_output}
        return results

Run network.

In [9]:
PatternInterpolationTrial().run(seed=1, p_seed=1, len_train=50, n_epoch=1, data_dir='debug_batch', data_format="npz")

running PatternInterpolationTrial#20190531-172135-17c349ee


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Run Number:  2
Testing with 80% Training Data


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 1
n_dims = 2
len_train = 50
n_epoch = 1
p_seed = 1
output_aimless000 = {'time': array(['1499781415', '1499781416', '1499781416', ..., '1499781451',
       '1499781451', '1499781451'], dtype='<U10'), 'output': array([[0.9156931 ],
       [0.92122197],
       [0.92272076],
       [0.92277615],
       [0.92221404],
       [0.91730525],
       [0.91462835],
       [0.91360359],
       [0.91435359],
       [0.91513996],
       [0.91849888],
       [0.92358978],
       [0.93195326],
       [0.94724171],
       [0.96296589],
       [0.97702928],
       [0.98973714],
       [1.00315645],
       [1.01894505],
       [1.02998566],
       [1.03188346],
       [1.01030723],
       [0.95027908],
       [0.89528649],
       [0.85455629],
       [0.81373347],
       [0.79388472],
       [0.77026436],
       [0.75459324],
       [0.79177113],
       [0.79008407],
       [0.79342935],
       [0.79050335],
       [0.79020804],
       [0.78195944],
       [0.77171577],
       [0.73512747],
       

{'output_goal_train': {'time': array(['1499435521', '1499435521', '1499435521', '1499435521',
         '1499435521', '1499435521', '1499435521', '1499435521',
         '1499435521', '1499435522', '1499435522', '1499435522',
         '1499435522', '1499435522', '1499435522', '1499435522',
         '1499435522', '1499435522', '1499435522', '1499435522',
         '1499435522', '1499435522', '1499435522', '1499435522',
         '1499435523', '1499435523', '1499435523', '1499435523',
         '1499435523', '1499435523', '1499435523', '1499435523',
         '1499435523', '1499435523', '1499435523', '1499435523',
         '1499435523', '1499435523', '1499435523', '1499435524',
         '1499435524', '1499435524', '1499435524', '1499435524',
         '1499435524', '1499435524', '1499435524', '1499435524',
         '1499435524', '1499435524', '1499435524', '1499435524',
         '1499435524', '1499435524', '1499435525', '1499435525',
         '1499435525', '1499435525', '1499435525', '149943552

In [ ]:
#for seed in range(20):
#    PatternInterpolationTrial().run(seed=seed, p_seed=seed, len_train=10, n_epoch=1, data_dir='theta15')
#    clear_output()